Welcome to the colab notebook for [GPTNeo](https://github.com/EleutherAI/GPTNeo) - a fully open source implementation of GPT like models for mesh-tensorflow by [EleutherAI](eleuther.ai).

Our library provides training and inference for GPT models up to GPT3 sizes on both TPUs and GPUs. 

In this notebook we walk you through TPU training (or finetuning!) and sampling using the freely available colab TPUs.

If you find our repo useful, come join [our discord](https://discord.gg/BK2v3EJ) and say hi! 😬

Before we get going - make sure you are running this notebook with a TPU available. Go to Runtime -> Change Runtime Type and select 'TPU' under hardware accelerator.




In [ ]:
#@title Setup
%tensorflow_version 1.x
!git clone https://github.com/EleutherAI/GPTNeo
%cd GPTNeo
!pip3 install -q -r requirements.txt

First, we need to download and tokenize a dataset - you can choose from:

*   OpenWebText - an opensource clone of OpenAI's WebText dataset, the original training data of GPT2.

*   YoutubeSubtitles - a dataset of subtitles scraped from youtube videos.

* Hackernews - comments scraped from hackernews

* NIHExporter - Data relating to various projects from the national institute of health.

All these datasets are from EleutherAI's side project - [The Pile™](https://github.com/EleutherAI/The-Pile) - an effort to gather a general purpose, diverse and open source plain text dataset large enough to train 1T+ parameter language models.

Even the smallest datasets are fairly large files, so this step will likely take a while. Select a dataset in the next cell, then run the next two cells, and go grab a snack and a cup of tea 😊

Alternatively, you can provide your own dataset in the form of a folder or gzip archive of .txt files. Simply select 'Custom' below and follow input the path to your data and the name of your dataset when prompted.

In [ ]:
# Select a Dataset:
dataset = 'HackerNews' #@param ["OpenWebText", "YoutubeSubtitles", "HackerNews", "NIHExporter", "Custom"]

In [ ]:
# Download Selected Dataset, or enter details of custom data:
import os

if dataset == 'OpenWebText':
  !gdown https://drive.google.com/uc?id=1EA5V0oetDCOke7afsktL_JDQ-ETtNOvx
  !tar xf openwebtext.tar.xz
  dataset_path = "openwebtext"
  dataset_name = dataset_path
  out_name = dataset_name + "_tokenized"
elif dataset == 'YoutubeSubtitles':
  os.makedirs('data', exist_ok=True)
  !wget wget https://eaidata.bmk.sh/data/yt_subs.jsonl.zst -O data/yt_subs.jsonl.zst
  dataset_path = 'data'
  dataset_name = 'ytsubs'
  out_name = dataset_name + "_tokenized"
elif dataset == 'HackerNews':
  os.makedirs('data', exist_ok=True)
  !wget https://eaidata.bmk.sh/data/hn.tar.gz -O data/hn.tar.gz
  dataset_path = 'data'
  dataset_name = 'hackernews'
  out_name = dataset_name + "_tokenized"
elif dataset == "NIHExporter":
  os.makedirs('data', exist_ok=True)
  !gdown https://drive.google.com/uc?id=11mO-0LuL2YeKoqqWXyHPHf3d2ODnjVPP
  dataset_path = 'data'
  os.system('mv NIH_ExPORTER_awarded_grant_text.jsonl.zst ./data')
  dataset_name = 'nihexporter'
  out_name = dataset_name + "_tokenized"
elif dataset == "Custom":
  dataset_path = input('Enter the path to the folder containing your data: ')
  dataset_name = input('Enter the name of your dataset: ')
  out_name = dataset_name + "_tokenized"
else:
  raise NotImplementedError('please select from available options: ["OpenWebText", "YoutubeSubtitles", "HackerNews", "NIHExporter", "Custom"]')



In [ ]:
# Tokenize Data:
!python data/create_tfrecords.py --mode documents --input_dir /content/GPTNeo/$dataset_path --name $dataset_name --output_dir $out_name --write_dataset_config

# Train on TPU

To train on TPUs we need to store our data on a google cloud bucket - as TPUs can't read from local filesystems.

You can set up a bucket by signing up for a free trial here: https://console.cloud.google.com/

Make a bucket at https://console.cloud.google.com/storage and come back when that's done.

The next cell sets up google authentication and gives the notebook read and write access to your bucket.


In [ ]:
from google.colab import auth
auth.authenticate_user()
!gcloud init

Now copy the tokenized data over to your google cloud bucket

In [ ]:
path_to_cloud_bucket = 'gs://path_to_your_bucket/datasets/' #@param {type:"string"}

In [ ]:
# copy the data to your bucket
copy_loc = path_to_cloud_bucket + dataset_name
!gsutil cp -r /content/GPTNeo/$out_name $copy_loc
!gsutil ls $path_to_cloud_bucket

Before starting training - you'll need to edit your dataset & model configs to point to your buckets / data

*   First change the writefile path to point to your chosen dataset - e.g `%%writefile configs/dataset_configs/ytsubs.json`
*   Change the "path" field to point to your cloud bucket location - e.g `gs://neo_lmdatasets/datasets/ytsubs_*.tfrecords`
* In the second cell (the model config) change "model_path" to the location in your cloud bucket you want the model weights to be saved to.
* Change the first item in the "datasets" list to the name of your chosen dataset. (the filename minus .json in ./configs/dataset_configs)
* Once you've made the edits, then run the cells to overwrite the existing files.




In [ ]:
%%writefile configs/dataset_configs/nihexporter.json

{
  "path": "gs://your_bucket_name/datasets/nihexporter/nihexporter_*.tfrecords",
  "eval_path": "",
  "n_vocab": 50256,
  "tokenizer_is_pretrained": true,
  "tokenizer_path": "gpt2",
  "eos_id": 50256,
  "padding_id": 50257
}


The model below is identical to our pretrained GPT2 model [TODO: add link]. You can finetune from this model by downloading the weights [TODO: add link] and copying them to your specified "model_path" below. 

If no previous model is found in "model_path", the model will start training from scratch.

If you want to use a smaller model, you can modify any of the config files in ../configs/ ending in _8.json, all of which are designed to train on tpu-v8s.

For a more detailed breakdown on what each item in the configuration file means - please read through our training and config guides in our [github README](https://github.com/EleutherAI/GPTNeo#training-guide). 

In [ ]:
%%writefile configs/colab_XL.json

{
    "n_head": 32,
    "n_vocab": 50260,
    "embed_dropout": 0,
    "lr": 0.0002,
    "lr_decay": "cosine",
    "warmup_steps": 3000,
    "beta1": 0.9,
    "beta2": 0.95,
    "epsilon": 1e-8,
    "opt_name": "adam",
    "weight_decay": 0.1,
    "train_batch_size": 256,
    "attn_dropout": 0,
    "train_steps": 572300,
    "eval_steps": 0,
    "predict_steps": 1,
    "res_dropout": 0,
    "eval_batch_size": 64,
    "predict_batch_size": 1,
    "iterations": 100,
    "n_embd": 2048,
    "datasets": [["edit_this", 21, "documents_random", 1.0]],
    "model": "GPT",
    "model_path": "gs://your_bucket_name/models/GPT3_XL",
    "n_ctx": 2048,
    "n_layer": 24,
    "scale_by_depth": true,
    "scale_by_in": false,
    "attention_types" :  [[["global"],24]],
    "mesh_shape": "x:4,y:2",
    "layout": "intermediate_expanded:x,heads:x,vocab:x,memory_length:y,embd:y",
    "activation_function": "gelu",
    "recompute_grad": true,
    "gradient_clipping": 1.0,
    "tokens_per_mb_per_replica": 2048
}

If everything's set up correctly, you can now run the main.py function to start training!

In [ ]:
!python3 main.py --model colab_XL --steps_per_checkpoint 500 --tpu colab

## Sample from your model:

Once training is finished, you can run the same command with the --predict flag to sample from your model.

To pass in a prompt, save it to a .txt file, and pass in the name of the file with the --prompt flag.

use the cell below to enter your prompt, and run it to save it to example_prompt.txt.

In [ ]:
%%writefile example_prompt.txt
In a shocking finding, scientists discovered a herd of unicorns living in a remote,
previously unexplored valley, in the Andes Mountains. Even more surprising to the
researchers was the fact that the unicorns spoke perfect English.

In [ ]:
!python3 main.py --model colab_XL_ --steps_per_checkpoint 500 --tpu colab --predict --prompt example_prompt.txt